## 1.1 Extração do Texto das Provas do ENEM 2020 - 2023

* **Input**: Local da Prova em formato PDF + Se questões são maiúsculas ou não

* **Output**: CSV com as colunas -> *numero_questao*, *enunciado* e *alternativas*

In [1]:
import pymupdf
import re
import csv
import pandas as pd

In [2]:
# Definição de Variáveis de Input e Output
year = 2023
question_caps = True

input_path = f"../data/provas/ENEM_{year}.pdf"
output_path = f"../data/extracted_questions/enem_{year}.csv"

In [3]:
# Extraindo texto do ENEM
document = pymupdf.open(input_path)
total_text = ""
keyword_found = False

for page_num, page in enumerate(document.pages(2), start=0):
    text = page.get_text()
    
    if keyword_found:
        total_text += text + "\n"
    elif "CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS" in text.upper():
        total_text += text + "\n"
        keyword_found = True

In [4]:
total_text

'20\n–CH • 1º DIA • CADERNO 1 • AZUL–\nCIÊNCIAS HUMANAS E SUAS TECNOLOGIAS\nQuestões de 46 a 90\nQUESTÃO 46\t\nFotografia da avó bordada\nSCARELI, G. A máquina de costura e os fios da memória. Revista Brasileira \nde Pesquisa (Auto)Biográfica, n. 18, maio-ago. 2021.\nA definição de Sertão descrita no bordado associa esse \nrecorte espacial a \nA\t\nA\t percursos e roteiros turísticos.\nB\t\nB\t trajetos e movimentos holísticos.\nC\t\nC\t vivências e itinerários socioafetivos.\nD\t\nD\t fronteiras e demarcações territoriais.\nE\t\nE\t profissões e interesses econômicos.\nQUESTÃO 47\t\nFelizes tempos eram esses! As moças iam à missa de \nmadrugada. De dia ninguém as via e se alguma, em dia \nde festa, queria passear com a avó ou a tia, havia de ir de \ncadeirinhas. Bem razão têm os nossos velhos de chorar \npor esses tempos, em que as filhas não sabiam escrever, \ne por isso não mandavam nem recebiam bilhetinhos.\nNovo Correio de Modas, 1853, apud DONEGÁ, A. L. Publicar ficção em \nmeado

In [5]:
# Remover o rodapé
rodape_pattern = r"\s+•\s+2º\s+DIA\s+•\s+CADERNO\s+7\s+•\s+AZUL"
total_text = re.sub(rodape_pattern, "", total_text)

# Remover códigos do rodapé, por exemplo: *020325AZ5*
total_text = re.sub(r"\*\w+\*", "", total_text)

# Remover linhas que contenham apenas dígitos (possíveis números de página)
total_text = re.sub(r"\n\s*\d+\s*\n", "\n", total_text)
total_text = re.sub(r"\n\s*\d+\s*$", "", total_text)

# Dividindo o texto em blocos de questões
question_blocks = re.split(r"QUESTÃO\s+" if question_caps else "Questão\s+", total_text)
question_blocks = [block.strip() for block in question_blocks if block.strip()]

results = []

for block in question_blocks:
    # Extraindo número da questão
    m = re.match(r"(\d+)", block)
    if not m:
        continue 
    num_quest = m.group(1)
    
    content = block[m.end():].strip()
    
    # Separando enunciado e alternativas.
    parts = re.split(r"\nA\s*\n", content, maxsplit=1)
    if len(parts) == 2:
        enunciado = parts[0].strip()
        alt_text = "A\n" + parts[1].strip()
    else:
        enunciado = content
        alt_text = ""
    
    # Extraindo alternativas
    alt_regex = r"([A-E])\s+[A-E]\s+(.*?)(?=(?:\n[A-E]\s+[A-E]\s+)|\Z)"
    alt_raw = re.findall(alt_regex, alt_text, flags=re.DOTALL)
    
    # Formatando alternativas como "A: texto"
    alternativas = "; ".join([f"{letter}: {text.strip()}" for letter, text in alt_raw])
    
    results.append({
        "numero_questao": num_quest,
        "enunciado": enunciado,
        "alternativas": alternativas
    })

In [11]:
# Salvando em CSV
with open(output_path, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["numero_questao", "enunciado", "alternativas"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in results[1:]:
        writer.writerow(row)

print("Extração Finalizada")

Extração Finalizada


In [12]:
df = pd.read_csv(output_path)
df.head()

,numero_questao,enunciado,alternativas
0,46,"Fotografia da avó bordada\nSCARELI, G. A máqui...",A: percursos e roteiros turísticos.; B: trajet...
1,47,Felizes tempos eram esses! As moças iam à miss...,A: Hábitos de leitura e mobilidade regional.; ...
2,48,"No Cerrado, o conhecimento local está sendo ca...","A: cerco aos camponeses, inviabilizando a manu..."
3,49,TEXTO I\nCom uma população de 25 milhões de ha...,A: Promover a ocupação rural.; B: Favorecer a ...
4,50,Superar a história da escravidão como principa...,A: recuperação do tradicionalismo.; B: estímul...


## Enem Digital (2021)

In [20]:
# Definição de Variáveis de Input e Output
year = 2021

input_path = f"../data/provas/ENEM_{year}.pdf"
output_path = f"../data/extracted_questions/enem_{year}.csv"

In [21]:
# Extraindo texto do ENEM
document = pymupdf.open(input_path)
total_text = ""

for page in document.pages(53,98):
    text = page.get_text()
    total_text += text + "\n"

In [22]:
# Padrão do ENEM Digital
question_pattern = r"Questão\s+(\d+)\s+-\s+Ciências Humanas e suas Tecnologias(.*?)(?=Questão\s+\d+\s+-\s+Ciências Humanas e suas Tecnologias|$)"
question_matches = re.findall(question_pattern, total_text, flags=re.DOTALL)

results = []

# Padrão de formatação das alternativas do ENEM Digital
alt_regex = r"\n(?:\d+\n)*([A-E])\n([^\n]+)"

for num_quest, block in question_matches:
    content = block.strip()

    alt_raw = re.findall(alt_regex, content)

    enunciado = content
    alt_match = re.search(alt_regex, content)
    if alt_match:
        enunciado = content[: alt_match.start()].strip()

    # Formatando alternativas como "A: texto"
    alternativas = "; ".join([f"{letter}: {text.strip()}" for letter, text in alt_raw])

    results.append(
        {
            "numero_questao": num_quest,
            "enunciado": enunciado,
            "alternativas": alternativas,
        }
    )

In [23]:
# Salvando em CSV
with open(output_path, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["numero_questao", "enunciado", "alternativas"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in results:
        writer.writerow(row)

print("Extração Finalizada")

Extração Finalizada


In [24]:
df = pd.read_csv(output_path)
df.head()

,numero_questao,enunciado,alternativas
0,46,"Seu turno de trabalho acabou, você já está em ...",A: proteção da vida privada.; B: ampliação de ...
1,47,Quando a taxa de remuneração do capital excede...,A: ideologia do mérito.; B: direito de nascime...
2,48,"Atualmente, o Programa de Melhoramento ""Uvas d...",A: espécies nativas ameaçadas.; B: cadeias eco...
3,49,TEXTO I\nPortadoras de mensagem espiritual do ...,A: ampliação do comércio de imagens sacras.; B...
4,50,"TEXTO I\nEm 2016, foram gerados 44,7 milhões d...",A: Exercício pleno da cidadania.; B: Divisão i...
